In [1]:
from rs_graph.modeling import predict_author_dev_matches

eva_github = {
    "username": "evamaxfield",
    "name": "Eva Maxfield Brown",
    "email": "None",
}
eva_author = {
    "name": "Eva Maxfield Brown",
}
nic_github = {
    "username": "nniiicc",
    "name": "Nic Weber",
    "email": "None",
}

predictions = predict_author_dev_matches(
    [eva_github, nic_github],
    [eva_author, eva_author],
)
predictions

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['match', 'no-match']

In [2]:
import pandas as pd
from tqdm import tqdm

from rs_graph.data import (
    load_author_contributions_dataset,
    load_repo_contributors_dataset,
)

# Load the two datasets
devs = load_repo_contributors_dataset()
authors = load_author_contributions_dataset().sample(100)

print(devs.shape)
print(authors.shape)

(15731, 8)
(100, 4)


In [3]:
processed_authors = []
repo_to_author_id_lut = {}
for _, author in authors.iterrows():
    repos = [c["repo"] for c in author.contributions]

    for repo in repos:
        if repo not in repo_to_author_id_lut:
            repo_to_author_id_lut[repo] = []

        repo_to_author_id_lut[repo].append(author.author_id)

    processed_authors.append({
        "name": author["name"],
        "author_id": author.author_id,
    })

processed_authors = pd.DataFrame(processed_authors)
processed_authors.sample(3)

,name,author_id
96,D. Larremore,2900564
73,E. Toyserkani,1714046
28,M. Dobias,46362392


In [4]:
# Create dataframe of dev username, name, email, repos, and co_contributors
processed_devs = []
repo_to_dev_lut = {}
for dev, group in devs.groupby("username"):
    repos = group["repo"].tolist()

    for repo in repos:
        if repo not in repo_to_dev_lut:
            repo_to_dev_lut[repo] = []

        repo_to_dev_lut[repo].append(dev)

    processed_devs.append({
        "username": dev,
        "name": group.iloc[0]["name"],
        "email": group.iloc[0]["email"],
    })

processed_devs = pd.DataFrame(processed_devs)
processed_devs.sample(3)

,username,name,email
121,AlexMikes,None,None
11437,wouterpeere,Wouter Peere,None
6018,hudajkhan,None,None


In [5]:
# Create comparison dataframe of devs and authors in the same repo
repo_and_dev_author_comparisons = []
for repo, dev_usernames in tqdm(repo_to_dev_lut.items()):
    if repo not in repo_to_author_id_lut:
        continue

    author_ids = repo_to_author_id_lut[repo]

    for dev_username in dev_usernames:
        for author_id in author_ids:
            dev_details = processed_devs.loc[
                processed_devs["username"] == dev_username
            ].iloc[0].to_dict()
            author_details = processed_authors.loc[
                processed_authors["author_id"] == author_id
            ].iloc[0].to_dict()

            # Append
            repo_and_dev_author_comparisons.append({
                "repo": repo,
                "dev_details": dev_details,
                "author_details": author_details,
            })

repo_and_dev_author_comparisons = pd.DataFrame(repo_and_dev_author_comparisons)
repo_and_dev_author_comparisons.sample(3)

100%|██████████| 2610/2610 [00:00<00:00, 2667.21it/s]


,repo,dev_details,author_details
206,https://github.com/resibots/limbo,"{'username': 'chrismile', 'name': 'Christoph N...","{'name': 'Federico Allocati', 'author_id': '75..."
691,https://github.com/ornladios/ADIOS2,"{'username': 'philip-davis', 'name': 'Philip D...","{'name': 'Keichi Takahashi', 'author_id': '193..."
57,https://github.com/michellab/BioSimSpace,"{'username': 'msuruzhon', 'name': 'Miroslav Su...","{'name': 'F. Gervasio', 'author_id': '2407694'}"


In [7]:
repo_and_dev_author_comparisons["predictions"] = predict_author_dev_matches(
    repo_and_dev_author_comparisons["dev_details"].tolist(),
    repo_and_dev_author_comparisons["author_details"].tolist(),
)
repo_and_dev_author_comparisons["predictions"].value_counts()

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

no-match    968
match        56
Name: predictions, dtype: int64

In [8]:
repo_and_dev_author_comparisons.loc[
    repo_and_dev_author_comparisons.predictions == "match"
][["dev_details", "author_details"]]

,dev_details,author_details
158,"{'username': 'tonielgeitani', 'name': 'Toni El...","{'name': 'Toni El Geitani', 'author_id': '2006..."
162,"{'username': 'bryanlimy', 'name': 'Bryan M. Li...","{'name': 'Bryan M. Li', 'author_id': '48218631'}"
167,"{'username': 'AnBucquet', 'name': 'BUCQUET Ant...","{'name': 'Anthime Bucquet', 'author_id': '1999..."
198,"{'username': 'PacoCosta', 'name': 'Francisco C...","{'name': 'Francisco Costa', 'author_id': '2150..."
209,"{'username': 'fedeallocati', 'name': 'Federico...","{'name': 'Federico Allocati', 'author_id': '75..."
260,"{'username': 'steven-murray', 'name': 'Steven ...","{'name': 'S. Murray', 'author_id': '47426998'}"
271,"{'username': 'Garyfallidis', 'name': 'Elefther...","{'name': 'Eleftherios Garyfallidis', 'author_i..."
299,"{'username': 'CKrawczyk', 'name': 'Coleman Kra...","{'name': 'Coleman Krawczyk', 'author_id': '984..."
320,"{'username': 'Carlitosh', 'name': 'Carlos Hern...","{'name': 'Carlos H. Hernandez', 'author_id': '..."
351,"{'username': 'emdupre', 'name': 'Elizabeth DuP...","{'name': 'E. Dupre', 'author_id': '33660027'}"
